In [1]:
import numpy as np
import random

In [2]:
class GameState():

    def __init__(self, playerleft = 1, playerright = 1, oppleft = 1, oppright = 1, turn = 0):
        self.player = {"l": playerleft, "r": playerright}
        self.opponent = {"l": oppleft, "r": oppright}
        self.turn = turn # 0 for player, 1 for opponent
        self.turn_number = 0
    
    def __str__(self):
        printleft = self.player["l"]
        printright = self.player["r"]
        printoppleft = self.opponent["l"]
        printoppright = self.opponent["r"]
        
        
        if printleft == 0:
            printleft = "~"
        
        if printright == 0:
            printright = "~"
            
        if printoppleft == 0:
            printoppleft = "~"
            
        if printoppright == 0:
            printoppright = "~"
        
        
        return "Player: " + str(printleft) + " " + str(printright) + " Opponent: " + str(printoppleft) + " " + str(printoppright)
    
    def hand_order(self):
        if self.player["l"] > self.player["r"]:
            self.player["l"], self.player["r"] = self.player["r"], self.player["l"]
        if self.opponent["l"] > self.opponent["r"]:
            self.opponent["l"], self.opponent["r"] = self.opponent["r"], self.opponent["l"]
    
    def play_turn(self, string):
        if string[0] == "a":
            return self.play_attack_turn(string[1], string[2])
        elif string[0] == "s":
            return self.play_shift_turn(int(string[1:]))
        elif string[0] == "r":
            vmoves = self.get_valid_moves()
            move = random.choice(vmoves)
            print("Move:", move)
            return self.play_turn(move)
    
    def play_attack_turn(self, turnhand, receivehand):
        
        assert turnhand in ["l", "r"], "Invalid hand"
        assert receivehand in ["l", "r"], "Invalid hand"
        
        
        if self.turn == 0:
            if self.player[turnhand] == 0:
                print("Cannot attack with an empty hand!")
                return False

            if self.opponent[receivehand] == 0:
                print("Cannot hit an empty hand!")
                return False
            
            self.opponent[receivehand] = (self.player[turnhand] + self.opponent[receivehand]) % 5
                    
        elif self.turn == 1:
            if self.opponent[turnhand] == 0:
                print("Cannot attack with an empty hand!")
                return False

            if self.player[receivehand] == 0:
                print("Cannot hit an empty hand!")
                return False
            
            self.player[receivehand] = (self.player[receivehand] + self.opponent[turnhand]) % 5
        
        self.turn = 1 - self.turn
        self.turn_number += 1
        return True
    
    def play_shift_turn(self, torightcount):
        
        
        if self.turn == 0: 
            tempr = self.player["r"] + torightcount
            templ = self.player["l"] - torightcount
            
            if (tempr < 0) or (templ < 0) or (tempr >= 5) or (templ >= 5):
                print("Invalid move!")
                return False
            else:
                self.player["r"] = tempr
                self.player["l"] = templ
            
        elif self.turn == 1:
            tempr = self.opponent["r"] + torightcount
            templ = self.opponent["l"] - torightcount
            
            if (tempr < 0) or (templ < 0) or (tempr >= 5) or (templ >= 5):
                print("Invalid move!")
                return False
            else:
                self.opponent["r"] = tempr
                self.opponent["l"] = templ
        
        self.turn = 1 - self.turn
        self.turn_number += 1
        return True
    
    def is_game_over(self):
        if self.player["l"] == 0 and self.player["r"] == 0:
            return "opponent"
        elif self.opponent["l"] == 0 and self.opponent["r"] == 0:
            return "player"
        elif self.turn_number >= 100:
            return "limit"
        else:
            return 0
    
    def get_valid_moves(self):
        
        vmoves = []

        if self.turn == 0:
            
            if self.player["l"] != 0:
                if self.opponent["l"] != 0:
                    vmoves.append("all")
                if self.opponent["r"] != 0:
                    vmoves.append("alr")
                
                for i in np.arange(1, self.player["l"] + 1):
                    if self.player["r"] + i < 5:
                        vmoves.append("s" + str(i))
                
            if self.player["r"] != 0:
                if self.opponent["l"] != 0:
                    vmoves.append("arl")
                if self.opponent["r"] != 0:
                    vmoves.append("arr")
    
                for i in range(1, self.player["r"] + 1):
                    if self.player["l"] + i < 5:
                        vmoves.append("s" + str(-i))
            
            d = self.player["r"] - self.player["l"]
            if d != 0:
                vmoves.remove("s" + str(-d))
            
        elif self.turn == 1:
            if self.opponent["l"] != 0:
                if self.player["l"] != 0:
                    vmoves.append("all")
                if self.player["r"] != 0:
                    vmoves.append("alr")
                
                for i in np.arange(1, self.opponent["l"] + 1):
                    if self.opponent["r"] + i < 5:
                        vmoves.append("s" + str(i))
                
            if self.opponent["r"] != 0:
                if self.player["l"] != 0:
                    vmoves.append("arl")
                if self.player["r"] != 0:
                    vmoves.append("arr")
    
                for i in range(1, self.opponent["r"] + 1):
                    if self.opponent["l"] + i < 5:
                        vmoves.append("s" + str(-i))

            d = self.opponent["r"] - self.opponent["l"]
            if d != 0:
                vmoves.remove("s" + str(-d))
            
        return vmoves

In [3]:

class Agent():
    def new_game(self):
        pass
    def reward(self, value):
        pass
    
class RandomAgent(Agent):
    def __init__(self):
        self.total_reward = 0
    
    def move(self, state):
        return random.choice(state.get_valid_moves())
    
    def reward(self, value):
        self.total_reward += value
        return value

In [4]:
def play_single_game(state, playerA, playerB, verbose = False):
    playerA.new_game()
    playerB.new_game()
    if verbose:
        print(state)
        
    while not state.is_game_over():
        move = playerA.move(state)
        if verbose:
            print("*" * 20)
            print("Player A move:", move)
        state.play_turn(move)
        if verbose:
            print(state)
        
        goc = state.is_game_over()
        if goc != 0:
            if goc == "limit":
                playerA.reward(0)
                playerB.reward(0)
                break
            playerA.reward(1)
            playerB.reward(-1)
            break
        
        move = playerB.move(state)
        if verbose:
            print("*" * 20)
            print("Player B move:", move)
        state.play_turn(move)
        if verbose:
            print(state)
        
        goc = state.is_game_over()
        if goc != 0:
            if goc == "limit":
                playerA.reward(0)
                playerB.reward(0)
                break
            playerA.reward(-1)
            playerB.reward(1)
            break
    return goc

In [5]:
def evaluateAgent(A, B, n = 1000, state = None):
    if state is None:
        state = GameState()
    
    Awin, Bwin, Draw = 0, 0, 0
    for game_no in range(n):
        winner = play_single_game(GameState(), randA, randB)
        if winner == "player":
            Awin += 1
        elif winner == "opponent":
            Bwin += 1
        elif winner == "limit":
            Draw += 1
    
    return Awin / n

In [134]:
randA = RandomAgent()
randB = RandomAgent()

play_single_game(GameState(), randA, randB)

'opponent'

In [129]:
randA.total_reward, randB.total_reward

(1, -1)

In [147]:
randA = RandomAgent()
randB = RandomAgent()

import time

Awin = 0
Bwin = 0
Draw = 0
t1 = time.time()
for i in range(10000):
    winner = play_single_game(GameState(), randA, randB)
    if winner == "player":
        Awin += 1
    elif winner == "opponent":
        Bwin += 1
    elif winner == "limit":
        Draw += 1
t2 = time.time()
print(t2-t1)

1.1627719402313232


In [148]:
randA.total_reward

-247

In [149]:
randB.total_reward

247

In [150]:
Awin

4865

In [151]:
Bwin

5112

In [152]:
Draw

23

In [53]:
gstest = GameState(4, 3, 1, 4)

In [54]:
gstest.get_valid_moves()

['all', 'alr', 's2', 's3', 's4', 'arl', 'arr', 's-1', 's-2', 's-3']

In [55]:
gstest.play_turn("s4")

Invalid move!


False